In [1]:
%run /content/AnnModel1.ipynb

In [2]:
# 메서드 정의
def multiple_main(epoch_count = 10, mb_size = 10, report=1, train_ratio = 0.6):
    multiple_load_dataset()
    init_param()
    train_and_test(epoch_count,mb_size,report,train_ratio)

In [3]:
# 메서드 정의
def multiple_load_dataset():
    with open('.../mulit_classification_data.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        rows = []
        for row in csvreader:
            rows.append(row)

    global data, input_cnt, output_cnt

    input_cnt, output_cnt = 27, 7
    data = np.asarray(rows, dtype='float32')

In [4]:
# 메서드 정의
def forward_postproc(y_hat, y_real):
    entropy  = softmax_cross_entropy(y_real, y_hat)
    loss     = np.mean(entropy)
        
    return loss, [y_real, y_hat, entropy]

In [5]:
# 메서드 정의
def softmax_cross_entropy(y_real, y_hat):

    probs = softmax(y_hat)
    
    #기존 확률값에 아주 작은 양수를 더해 log함수의 폭주 방지
    return - np.sum(y_real * np.log(probs+1.0e-10), axis=1)

In [6]:
# 메서드 정의
def softmax(x):

    max_elem = np.max(x, axis = 1)
    
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    
    sum_exp = np.sum(exp, axis = 1)
    
    probs = (exp.transpose()/sum_exp).transpose()

    return probs

In [7]:
# 메서드 정의
def backprop_postproc(aux_pp):
    y_real, y_hat, entropy = aux_pp

    g_loss_entropy = 1.0/np.prod(entropy.shape)
    g_entropy_output = softmax_cross_entropy_with_derv(y_real,y_hat)
    
    G_output = g_entropy_output * g_loss_entropy 
    
    return G_output

In [8]:
# 메서드 정의
def softmax_cross_entropy_with_derv(y_real,y_hat):
    
    return softmax(y_hat) - y_real

In [9]:
# 메서드 정의
def eval_accuracy(y_hat,y_real):
    estimate = np.argmax(y_hat,axis=1)
    answer = np.argmax(y_real,axis=1)
    
    correct = np.equal(estimate, answer)

    return np.mean(correct)